In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score


In [7]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
df=pd.read_csv("Hitters.csv")
df=df.dropna()

dms=pd.get_dummies(df[['League', 'Division','NewLeague']])

y=df['Salary']
X_= df.drop(['Salary','League', 'Division','NewLeague'],axis=1).astype('float64')
X= pd.concat([X_, dms[['League_N', 'Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

#### Burada diğer yöntemlere göre standartlaştırma işlemi gerçekleştireceğiz. 
#### Bazı algoritmalar heterojen veri setlerinde iyi çalışırken bazıları homojen veri setlerinde iyi çalışır. 
#### YSA homojen veri setlerinde daha iyidir. Çünkü hataların optimizasyonu kısmında yapılan türevleme işlemlerinde ortaya çıkan aykırılıkların ortaya çıkardığı problemler olabilir. Bu yüzden standartlaştırma işlemi sonrası YSA kullanmak daha mantıklıdır. 

In [3]:
from sklearn.preprocessing import scale, StandardScaler


In [4]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)

In [5]:
X_test_scaled=scaler.transform(X_test)

In [8]:
from sklearn.neural_network import MLPRegressor

mlp_model=MLPRegressor().fit(X_train_scaled,y_train)

In [12]:
y_pred=mlp_model.predict(X_test_scaled)
y_pred

array([ 20.93616455,  36.11784637,  62.95833543,  12.67774117,
        21.99905052,   9.57922114,  15.74636452,  11.8757594 ,
        70.8146361 ,  36.85797283,  21.96587785,  71.78850512,
        18.51793205,  27.21384357,   8.37700755,  47.7007535 ,
        57.71102195,  13.64044309,  57.501992  ,   9.29469042,
         7.184367  ,  47.73968957,  14.40090489,  10.50313336,
        51.37068419,  16.16342242,  63.75800464,   9.12230803,
        24.90936936,  14.06521035,  15.05177456,  19.76366505,
         6.71848596,  20.09435322,  11.13564479,  36.38833841,
       121.42122435,   7.89764597,  17.14347397,  10.8056836 ,
        16.94986772,  10.08105811,  11.8591207 ,   6.75989212,
        77.90629255,  39.55636477,  13.47285122,  11.64905456,
         9.99967224,  32.32286216,  39.89012353,   9.36392545,
        31.50945335,   9.92484654,  12.78787969,  28.59631783,
         8.25992182,  38.80997943,  15.7461033 , 116.45298761,
       125.46158558,  15.74732893,  35.0759452 ,   6.46

In [13]:
np.sqrt(mean_squared_error(y_test,y_pred))

659.6451253807506

## Model Tuning

In [14]:
mlp_params={"alpha":[0.1,0.01,0.001,0.002,0.0001],
            "hidden_layer_sizes": [(10,2),(10,20), (5,5),(100,100)] }

In [24]:
mlp_cv_model=GridSearchCV(mlp_model, mlp_params, cv=5, verbose=2,n_jobs=-1).fit(X_train_scaled,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [25]:
mlp_cv_model.best_params_

{'alpha': 0.01, 'hidden_layer_sizes': (100, 100)}

In [27]:
mlp_tuned=MLPRegressor(alpha=0.01,hidden_layer_sizes=(100,100)).fit(X_train_scaled,y_train)

In [28]:
y_pred=mlp_tuned.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

365.1798941379399